<a href="https://colab.research.google.com/github/suryanshsinghrathod35-bit/suryanshsinghrathod35-bit/blob/main/second_popu_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import kagglehub
import os

path = kagglehub.dataset_download(
    "gauravkumar2525/country-population-and-growth-rate-analysis")

print("Path to dataset files:", path)

for file in os.listdir(path):
    if file.endswith(".csv"):
        csv_path = os.path.join(path, file)
        break

df = pd.read_csv(csv_path)

# Display the first 10 rows
df.head(10)


Path to dataset files: /kaggle/input/country-population-and-growth-rate-analysis


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Country,ISO3,Year,Population,Population Growth,Growth Rate (%),Decade
0,Afghanistan,AFG,1960,8996973,NaN,NaN,1960s
1,Afghanistan,AFG,1961,9169410,172437.0,1.916611,1960s
2,Afghanistan,AFG,1962,9351441,182031.0,1.985199,1960s
3,Afghanistan,AFG,1963,9543205,191764.0,2.050636,1960s
4,Afghanistan,AFG,1964,9744781,201576.0,2.112246,1960s
5,Afghanistan,AFG,1965,9956320,211539.0,2.170793,1960s
6,Afghanistan,AFG,1966,10174836,218516.0,2.194747,1960s
7,Afghanistan,AFG,1967,10399926,225090.0,2.212222,1960s
8,Afghanistan,AFG,1968,10637063,237137.0,2.280180,1960s
9,Afghanistan,AFG,1969,10893776,256713.0,2.413382,1960s


In [ ]:
df.tail() #shows dataframe from bottom

,Country,ISO3,Year,Population,Population Growth,Growth Rate (%),Decade
15404,Zimbabwe,ZWE,2014,13586681,236325.0,1.770178,2010s
15405,Zimbabwe,ZWE,2015,13814629,227948.0,1.677731,2010s
15406,Zimbabwe,ZWE,2016,14030390,215761.0,1.561830,2010s
15407,Zimbabwe,ZWE,2017,14236745,206355.0,1.470772,2010s
15408,Zimbabwe,ZWE,2018,14439018,202273.0,1.420781,2010s


In [ ]:
df.shape

(15409, 7)

In [ ]:
#cleaning the dataset
df.isnull().sum()

Country                0
ISO3                   0
Year                   0
Population             0
Population Growth    263
Growth Rate (%)      263
Decade                 0
dtype: int64

In [ ]:
#splitting of dataset
X = df.drop('Population', axis=1)
y = df['Population']

In [ ]:
#finding the numerical and categorical cols
num_cols = X.select_dtypes(exclude='object').columns
cat_cols = X.select_dtypes(include='object').columns

In [ ]:
print(num_cols)
print(cat_cols)

Index(['Year', 'Population Growth', 'Growth Rate (%)'], dtype='object')
Index(['Country', 'ISO3', 'Decade'], dtype='object')


In [ ]:
#splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,random_state=42)

In [ ]:
X_train.isnull().sum()

Country                0
ISO3                   0
Year                   0
Population Growth    194
Growth Rate (%)      194
Decade                 0
dtype: int64

In [ ]:
#filling in the Missing values
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(strategy='mean')
X_train_imputed = my_imputer.fit_transform(X_train[num_cols])
X_test_imputed = my_imputer.transform(X_test[num_cols])

X_train_imputed = pd.DataFrame(X_train_imputed, columns=num_cols, index=X_train.index)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=num_cols, index=X_test.index)

In [ ]:
#but since the whole data is categorical we gotta
# change it to numeric dataset first coz our model understands only
#numeric language
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

oh_cols_train = pd.DataFrame(ohe.fit_transform(X_train[cat_cols]))
oh_cols_test = pd.DataFrame(ohe.transform(X_test[cat_cols]))

# One-hot encoding removed index; put it back
oh_cols_train.index = X_train.index
oh_cols_test.index = X_test.index

# Optional: add proper column names
oh_cols_train.columns = ohe.get_feature_names_out(cat_cols)
oh_cols_test.columns = ohe.get_feature_names_out(cat_cols)

# Access the cleaned numerical columns (already imputed)
num_X_train = X_train_imputed
num_X_test = X_test_imputed

# Add one-hot encoded columns to numerical features
oh_X_train = pd.concat([num_X_train, oh_cols_train], axis=1)
oh_X_test = pd.concat([num_X_test, oh_cols_test], axis=1)


In [ ]:
# scaling it
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit on training data and transform both training & validation
X_train_scaled = scaler.fit_transform(oh_X_train)
X_test_scaled = scaler.transform(oh_X_test)

# Convert back to DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=oh_X_train.columns, index=oh_X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=oh_X_test.columns, index=oh_X_valid.index)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize model
lr_model = LinearRegression()

# Train (fit) the model
lr_model.fit(X_train_scaled, y_train)

# Predict on validation data
y_pred_lr = lr_model.predict(X_test_scaled)

# Evaluate performance
mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression Results:")
print("MAE:", mae_lr)
print("MSE:", mse_lr)
print("R²:", r2_lr)


Linear Regression Results:
MAE: 90079224.98818277
MSE: 3.5653915853016736e+16
R²: 0.9308873501330474


In [ ]:
from sklearn.linear_model import SGDRegressor

# Initialize model
sgd_model = SGDRegressor(random_state=42, max_iter=1000, alpha=0.0001)

# Train
sgd_model.fit(X_train_scaled, y_train)

# Predict
y_pred_sgd = sgd_model.predict(X_valid_scaled)

# Evaluate
mae_sgd = mean_absolute_error(y_test, y_pred_sgd)
mse_sgd = mean_squared_error(y_test, y_pred_sgd)
r2_sgd = r2_score(y_test, y_pred_sgd)

print("\nSGD Regressor Results:")
print("MAE:", mae_sgd)
print("MSE:", mse_sgd)
print("R²:", r2_sgd)



SGD Regressor Results:
MAE: 71172495.65937515
MSE: 3.273806473885049e+16
R²: 0.9365395258421143
